<span style="color:#888888">Copyright (c) 2014-2021 National Technology and Engineering Solutions of Sandia, LLC. Under the terms of Contract DE-NA0003525 with National Technology and Engineering Solutions of Sandia, LLC, the U.S. Government retains certain rights in this software.     Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:</span>

<span style="color:#888888">1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.</span>

<span style="color:#888888">2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.</span>

<span style="color:#888888">THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.</span>

# <span style="color:#0054a8">**Demo 4:**</span> <span style="color:#555555">Detecting Rendezvous with DBSCAN</span>

Using Tracktable's box-DBSCAN capabilities, we can detect trajectories that are traveling together for part or all of their journeys.

<details closed>
    <summary><b>Algorithm Details</b></summary>
    <br>
    First, we decide during which portion of the journeys we want to detect rendezvous behavior.  For instance, we can say from 40% to 60% of the trajectory length, so we will detect any trajectories that are together from 40% to 60% of their travels, or we could just leave this as the default of 0% to 100%, which will only detect trajectories that travel together during the entire duration of their journeys.
    <br>
    <br>
    For every trajectory, we create a <b><span style="color:#00add0">feature vector</span></b>, which we call the rendezvous feature vector, by taking a given number of equally-spaced points along the trajectory (during the portion we specified) and concatenating the longitude, latitude and timestamp from all of these points into a single vector.  By default, we use 10 equally-spaced points along the trajectory, making our feature vectors 30-dimensional.
    <br>
    <br>
    Using <b><span style="color:#00add0">box-DBSCAN</span></b>, Tracktable clusters trajectories together based on the similarity of their feature vectors.  Trajectories that travel together during the specified portion of their trajectories will occupy the same cluster.
</details>

In [ ]:
from tracktable.render.render_trajectories import render_trajectories
import tracktable.examples.tutorials.tutorial_helper as tutorial 
from tracktable.applications.cluster import *

## Import Trajectories

We will use some sample maritime data for this demo, obtained from BOEM/NOAA.$^1$

In [ ]:
trajectories = tutorial.get_trajectory_list('rendezvous')

## Filter Idle Subtrajectories

Rendezvous detection relies on comparing the temporospatial information at a given number of points equally-spaced along a portion of the trajectory to identify trajectories that travel together.  If a boat stays mostly in port and travels for a short time, our equally-spaced points may miss the traveling behavior.  Therefore, to make our rendezvous detection more effective, we delete portions of trajectories that are sitting idle, possibly splitting the trajectory into multiple trajectories if the trajectory moves, is idle, and then moves again.

In [ ]:
trimmed_trajectories = tutorial.split_trajectories(trajectories)

Let's also trim very short trajectories (less than 5km traveled) from our dataset.

In [ ]:
filtered_trajectories = tutorial.filter_trajectories(trimmed_trajectories, min_length=5)

## *Option 1:* Find trajectories that travel together from start to finish.

Now that we have a list of filtered trajectories, we can use Tracktable to create feature vectors for each, then use box-DBSCAN to cluster over those feature vectors.

By default, we create our feature vector using given number of equally-spaced points along the entire trajectory and concatenate the **longitude**, **latitude** and **timestamp** from all of these points into a single 30-dimensional vector.

In [ ]:
clusters = cluster_trajectories_rendezvous(filtered_trajectories)

The `clusters` dictionary contains cluster numbers as keys, and a list of trajectories in that cluster for each corresponding value.  These clusters and their sizes are printed above.

In our trajectory dataset, we have found one cluster, meaning one set of trajectories that was traveling together for their entire journeys.  We will render this cluster below.

In [ ]:
render_trajectories(clusters[1])

The trajectories that did not cluster (outliers) are stored under key `0` of the `clusters` dictionary.  Let's look at the first ten of these trajectories that are not traveling with any others.

In [ ]:
render_trajectories(clusters[0][:10])

## *Option 2:* Find trajectories that travel together for only a portion of their travel.

Alternatively, we can select a start and end fraction (proportional to trajectory length) that we wish to examine for rendezvous behavior.  In the example below, we look for trajectories that are traveling together from 60% to 80% of their travels.

In [ ]:
clusters = cluster_trajectories_rendezvous(filtered_trajectories, start_fraction=0.6, end_fraction=0.8)

Now we have found three clusters, meaning three sets of trajectories that are rendezvousing during the 60% to 80% portion of their trajectories.  Let's render each cluster below.

*Note:* The cluster that was found in the previous example, containing two trajectories that were traveling together during their entire journeys, will also be a cluster here, as they will also be traveling together during the 60% to 80% portion of their trajectories.

In [ ]:
render_trajectories(clusters[1])

In [ ]:
render_trajectories(clusters[2])

In [ ]:
render_trajectories(clusters[3])

<span style="color:gray">$^1$ Bureau of Ocean Energy Management (BOEM) and National Oceanic and Atmospheric Administration (NOAA). MarineCadastre.gov. *AIS Data for 2020.* Retrieved February 2021 from [marinecadastre.gov/data](https://marinecadastre.gov/data/).  Data is for June 4-6, 2020 martime traffic near Virginia Beach.</span>